# Monitoreo de Cambios Agrícolas con Sentinel-1 SAR
## Análisis Completo - Llanos Orientales de Colombia

### Resumen
Notebook consolidado que integra preparación de datos, preprocesamiento Sentinel-1, detección de cambios y visualización.

**Municipios:** 9 en Casanare y Meta  
**Metodología:** Canty et al. (2020), Conradsen et al. (2003)  
**Plataforma:** Google Earth Engine

## 1. Configuración del Entorno

### Instalación de Dependencias
```bash
pip install earthengine-api geemap geopandas fiona matplotlib seaborn pandas numpy plotly jupyter
```

In [ ]:
# Importar librerías
import ee
import geemap
import geopandas as gpd
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
from datetime import datetime, timedelta
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuración
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context('notebook', font_scale=1.1)
sns.set_palette('Set2')

Path('data').mkdir(exist_ok=True)
print('✓ Librerías cargadas')

In [ ]:
# Inicializar Google Earth Engine
try:
    ee.Initialize()
    print('✓ Earth Engine inicializado')
except:
    print('⚠ Ejecute: ee.Authenticate()')
    raise

## 2. Preparación de Datos y Área de Estudio

### 2.1. Configuración de Parámetros

In [ ]:
# Parámetros temporales
FECHA_INICIO = '2023-01-01'
FECHA_FIN = '2024-12-31'

# Períodos para análisis de cambios
REFERENCE_START = '2023-01-01'
REFERENCE_END = '2023-06-30'
TARGET_START = '2024-01-01'
TARGET_END = '2024-06-30'

# Ruta al GPKG
GPKG_PATH = '/home/famartinezal/Dropbox/Base/DANE_BASE_2023.gpkg'
LAYER_NAME = 'MGN_MPIO_POLITICO'

print(f'Período total: {FECHA_INICIO} a {FECHA_FIN}')
print(f'Referencia: {REFERENCE_START} a {REFERENCE_END}')
print(f'Análisis: {TARGET_START} a {TARGET_END}')

### 2.2. Carga de Municipios

In [ ]:
# Cargar capa de municipios
municipios_colombia = gpd.read_file(GPKG_PATH, layer=LAYER_NAME)

# Municipios objetivo
MUNICIPIOS_META = ['PUERTO LÓPEZ', 'CASTILLA LA NUEVA', 'SAN CARLOS DE GUAROA', 'CABUYARO']
MUNICIPIOS_CASANARE = ['TAURAMENA', 'YOPAL', 'AGUAZUL', 'NUNCHÍA', 'VILLANUEVA']

municipios = municipios_colombia[
    (municipios_colombia['dpto_cnmbr'].isin(['META', 'CASANARE'])) &
    (municipios_colombia['mpio_cnmbr'].isin(MUNICIPIOS_META + MUNICIPIOS_CASANARE))
].copy()

municipios = municipios.to_crs(epsg=4326)
print(f'✓ Municipios cargados: {len(municipios)}')
print(municipios[['dpto_cnmbr', 'mpio_cnmbr']].to_string(index=False))

### 2.3. Crear AOI para Earth Engine

In [ ]:
def gdf_to_ee_geometry(gdf):
    geom = gdf.geometry.unary_union
    if geom.geom_type == 'Polygon':
        return ee.Geometry.Polygon([list(geom.exterior.coords)])
    else:
        return ee.Geometry.MultiPolygon([list(p.exterior.coords) for p in geom.geoms])

aoi = gdf_to_ee_geometry(municipios)
centroide = municipios.geometry.unary_union.centroid
bounds = municipios.total_bounds

print(f'✓ AOI creada')
print(f'Centroide: {centroide.y:.4f}, {centroide.x:.4f}')

## 3. Preprocesamiento Sentinel-1

### 3.1. Definir Pipeline de Procesamiento

In [ ]:
def process_sentinel1(aoi, start, end, orbit=None):
    s1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(aoi).filterDate(start, end) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    
    if orbit:
        s1 = s1.filter(ee.Filter.eq('orbitProperties_pass', orbit))
    
    def to_dB(img):
        vv = ee.Image(10).multiply(img.select('VV').log10()).rename('VV')
        vh = ee.Image(10).multiply(img.select('VH').log10()).rename('VH')
        ratio = vv.subtract(vh).rename('VV_VH_ratio')
        return img.addBands([vv, vh, ratio]).copyProperties(img, ['system:time_start'])
    
    def speckle_filter(img):
        k = ee.Kernel.square(3, 'pixels')
        vv = img.select('VV').focal_median(k).rename('VV_filt')
        vh = img.select('VH').focal_median(k).rename('VH_filt')
        ratio = img.select('VV_VH_ratio').focal_median(k).rename('ratio_filt')
        return img.addBands([vv, vh, ratio])
    
    return s1.map(to_dB).map(speckle_filter)

print('✓ Pipeline definido')

### 3.2. Cargar y Filtrar Colección

In [ ]:
# Cargar colección
s1_col = process_sentinel1(aoi, FECHA_INICIO, FECHA_FIN)

# Seleccionar órbita con más imágenes
s1_asc = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
s1_desc = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))

n_asc = s1_asc.size().getInfo()
n_desc = s1_desc.size().getInfo()

if n_asc >= n_desc:
    s1_filtered = s1_asc
    orbit_type = 'ASCENDING'
else:
    s1_filtered = s1_desc
    orbit_type = 'DESCENDING'

print(f'Órbita seleccionada: {orbit_type}')
print(f'Imágenes disponibles: {s1_filtered.size().getInfo()}')

## 4. Detección de Cambios

### 4.1. Crear Composiciones

In [ ]:
# Composiciones de referencia y análisis
ref_comp = s1_filtered.filterDate(REFERENCE_START, REFERENCE_END).median().clip(aoi)
target_comp = s1_filtered.filterDate(TARGET_START, TARGET_END).median().clip(aoi)

n_ref = s1_filtered.filterDate(REFERENCE_START, REFERENCE_END).size().getInfo()
n_target = s1_filtered.filterDate(TARGET_START, TARGET_END).size().getInfo()

print(f'Composición referencia: {n_ref} imágenes')
print(f'Composición análisis: {n_target} imágenes')

### 4.2. Métodos de Detección

#### Método 1: Diferencias Temporales

In [ ]:
# Diferencias absolutas
diff_vv = target_comp.select('VV_filt').subtract(ref_comp.select('VV_filt')).rename('diff_VV')
diff_vh = target_comp.select('VH_filt').subtract(ref_comp.select('VH_filt')).rename('diff_VH')

# Magnitud de cambio
change_mag = diff_vv.pow(2).add(diff_vh.pow(2)).sqrt().rename('change_magnitude')

print('✓ Diferencias calculadas')

#### Método 2: Índice NDCV

In [ ]:
def calc_ndcv(band):
    ref_lin = ee.Image(10).pow(ref_comp.select(band).divide(10))
    target_lin = ee.Image(10).pow(target_comp.select(band).divide(10))
    return target_lin.subtract(ref_lin).abs().divide(target_lin.add(ref_lin))

ndcv_vv = calc_ndcv('VV_filt').rename('NDCV_VV')
ndcv_vh = calc_ndcv('VH_filt').rename('NDCV_VH')
ndcv_combined = ndcv_vv.add(ndcv_vh).divide(2).rename('NDCV')

# Máscara de cambio
change_mask = ndcv_combined.gt(0.3).rename('change_mask')

print('✓ NDCV calculado')

#### Método 3: Clasificación de Cambios

In [ ]:
# Umbrales
STRONG_THRESH = 3.0
MOD_THRESH = 1.5

# Clasificación
change_class = ee.Image(0).clip(aoi)
change_class = change_class.where(diff_vv.gt(STRONG_THRESH), 1)  # Aumento fuerte
change_class = change_class.where(diff_vv.lt(-STRONG_THRESH), 2)  # Disminución fuerte
change_class = change_class.where(
    diff_vv.gt(MOD_THRESH).And(diff_vv.lte(STRONG_THRESH)), 3
)  # Aumento moderado
change_class = change_class.where(
    diff_vv.lt(-MOD_THRESH).And(diff_vv.gte(-STRONG_THRESH)), 4
)  # Disminución moderada
change_class = change_class.rename('change_class')

print('✓ Clasificación completada')

## 5. Estadísticas por Municipio

In [ ]:
# Extraer estadísticas
stats_list = []

for idx, row in municipios.iterrows():
    geom = row.geometry
    if geom.geom_type == 'Polygon':
        ee_geom = ee.Geometry.Polygon([list(geom.exterior.coords)])
    else:
        ee_geom = ee.Geometry.MultiPolygon([list(p.exterior.coords) for p in geom.geoms])
    
    # Estadísticas de diferencia
    diff_stats = diff_vv.addBands(diff_vh).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=ee_geom,
        scale=10,
        maxPixels=1e9
    )
    
    # NDCV
    ndcv_mean = ndcv_combined.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=ee_geom,
        scale=10,
        maxPixels=1e9
    ).get('NDCV')
    
    # Área de cambio
    area_total = ee_geom.area().divide(10000)  # ha
    area_cambio = change_mask.multiply(ee.Image.pixelArea()).divide(10000).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=ee_geom,
        scale=10,
        maxPixels=1e9
    ).get('change_mask')
    
    stats = {
        'municipio': row['mpio_cnmbr'],
        'departamento': row['dpto_cnmbr'],
        'diff_VV': diff_stats.get('diff_VV'),
        'diff_VH': diff_stats.get('diff_VH'),
        'NDCV': ndcv_mean,
        'area_total_ha': area_total.getInfo(),
        'area_cambio_ha': ee.Number(area_cambio).getInfo(),
        'pct_cambio': ee.Number(area_cambio).divide(area_total).multiply(100).getInfo()
    }
    
    stats_list.append(stats)
    print(f'  Procesado: {row["mpio_cnmbr"]}')

stats_df = pd.DataFrame(stats_list).sort_values('pct_cambio', ascending=False)
print('\n' + stats_df.to_string(index=False))

## 6. Visualización

### 6.1. Mapa Interactivo

In [ ]:
Map = geemap.Map(center=[centroide.y, centroide.x], zoom=9)
Map.add_gdf(municipios, layer_name='Municipios', style={'fillOpacity': 0})

# Parámetros de visualización
vis_diff = {'min': -5, 'max': 5, 'palette': ['red', 'white', 'blue']}
vis_ndcv = {'min': 0, 'max': 0.6, 'palette': ['white', 'yellow', 'orange', 'red']}
vis_class = {'min': 0, 'max': 4, 'palette': ['gray', 'blue', 'red', 'lightblue', 'orange']}

Map.addLayer(diff_vv, vis_diff, 'Diferencia VV', True)
Map.addLayer(ndcv_combined, vis_ndcv, 'NDCV', True)
Map.addLayer(change_class, vis_class, 'Clasificación', False)

Map

### 6.2. Gráficos Estadísticos

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 10))

# Porcentaje de cambio
colors = ['#1f77b4' if d == 'CASANARE' else '#ff7f0e' for d in stats_df['departamento']]
axes[0].barh(range(len(stats_df)), stats_df['pct_cambio'], color=colors)
axes[0].set_yticks(range(len(stats_df)))
axes[0].set_yticklabels(stats_df['municipio'])
axes[0].set_xlabel('Porcentaje de Área con Cambio (%)')
axes[0].set_title('Detección de Cambios por Municipio', fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# Área en hectáreas
axes[1].barh(range(len(stats_df)), stats_df['area_cambio_ha'], color=colors)
axes[1].set_yticks(range(len(stats_df)))
axes[1].set_yticklabels(stats_df['municipio'])
axes[1].set_xlabel('Área con Cambio (hectáreas)')
axes[1].set_title('Área Total con Cambios Detectados', fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('data/cambios_municipios.png', dpi=300, bbox_inches='tight')
plt.show()
print('✓ Gráficos guardados')

## 7. Exportación de Resultados

In [ ]:
# Guardar estadísticas
stats_df.to_csv('data/estadisticas_cambios.csv', index=False)

# Guardar parámetros
params = {
    'fecha_inicio': FECHA_INICIO,
    'fecha_fin': FECHA_FIN,
    'reference_period': {'start': REFERENCE_START, 'end': REFERENCE_END},
    'target_period': {'start': TARGET_START, 'end': TARGET_END},
    'orbit_type': orbit_type,
    'n_images_ref': n_ref,
    'n_images_target': n_target,
    'strong_threshold': STRONG_THRESH,
    'moderate_threshold': MOD_THRESH
}

with open('data/parametros_analisis.json', 'w') as f:
    json.dump(params, f, indent=2)

# GeoPackage con resultados
municipios_results = municipios.merge(
    stats_df[['municipio', 'pct_cambio', 'area_cambio_ha', 'NDCV']],
    left_on='mpio_cnmbr',
    right_on='municipio',
    how='left'
)
municipios_results.to_file('data/municipios_resultados.gpkg', driver='GPKG')

print('✓ Resultados exportados en data/')

## Resumen y Conclusiones

### Productos Generados

1. ✓ Estadísticas de cambio por municipio (CSV)
2. ✓ Parámetros del análisis (JSON)
3. ✓ Capa espacial con resultados (GeoPackage)
4. ✓ Visualizaciones (PNG)
5. ✓ Mapa interactivo (geemap)

### Interpretación Agrícola

**Cambios detectados:**
- **Aumento de backscatter VV/VH**: Posible crecimiento vegetativo, emergencia de cultivos, inundación de campos (preparación arroz)
- **Disminución de backscatter**: Posible cosecha, senescencia, preparación de suelo, sequía
- **Alta variabilidad**: Rotación de cultivos, gestión agrícola activa

### Referencias Completas

Ver archivo `references.bib` para bibliografía en formato BibTeX.

**Referencias clave:**
- Canty et al. (2020): https://doi.org/10.3390/rs12010046
- Conradsen et al. (2003): https://doi.org/10.1109/TGRS.2002.808066
- Veloso et al. (2017): https://doi.org/10.1016/j.rse.2017.07.015